In [1]:
# Import Libraries
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import numpy as np
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import tqdm
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split

c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Specify transforms using torchvision.transforms as transforms
# library
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
# Load in each dataset and apply transformations using
# the torchvision.datasets as datasets library
train_set = datasets.ImageFolder("D:/learn_django_v3/x-ray/chest_xray/train/", transform = transformations)
val_set = datasets.ImageFolder("D:/learn_django_v3/x-ray/chest_xray/val/", transform = transformations)

In [4]:

# Specify transforms using torchvision.transforms as transforms
# library
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Load in each dataset and apply transformations using
# the torchvision.datasets as datasets library
train_set = datasets.ImageFolder("D:/learn_django_v3/x-ray/chest_xray/train/", transform = transformations)
val_set = datasets.ImageFolder("D:/learn_django_v3/x-ray/chest_xray/val/", transform = transformations)
# Put into a Dataloader using torch library
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size =128, shuffle=True)
# Get pretrained model using torchvision.models as models library
model = models.resnet18(pretrained=True)
# Turn off training for their parameters
for param in model.parameters():
    param.requires_grad = False


    


c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\rifat\miniconda3\envs\tf\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Get pretrained model using torchvision.models as models library
model = models.resnet18(pretrained=True)
# Turn off training for their parameters
for param in model.parameters():
    param.requires_grad = False

In [6]:
# Create new classifier for model using torch.nn as nn library
classifier_input = model.fc.in_features
num_labels = 2
classifier = nn.Sequential(nn.Linear(classifier_input, 100),
                            nn.ReLU(),
                            nn.Linear(100, 512),
                            nn.ReLU(),
                            nn.Dropout(p=0.5),
                            nn.Linear(512, 128),
                            nn.ReLU(),
                            nn.Dropout(p=0.5),
                            nn.Linear(128, 64),
                            nn.ReLU(),
                            nn.Dropout(p=0.5),
                            nn.Linear(64, num_labels),
                            nn.LogSoftmax(dim=1))
model.fc = classifier
# Find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_trainable_params:,} training parameters.')


11,353,574 total parameters.
177,062 training parameters.


In [7]:

# Define Optimizer and Loss Function
writer = SummaryWriter()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00021367223615921426)
def train(epoch):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(train_set)
    epoch_acc = running_corrects.double() / len(train_set)
    writer.add_scalar('training loss', epoch_loss, epoch)
    writer.add_scalar('training acc', epoch_acc, epoch)
    print(f'training loss: {epoch_loss:.4f}, acc {epoch_acc:.4f}')
    model_filename = f'model_epoch_{epoch}.pt'
    torch.save(model.state_dict(), model_filename)
    print(f'Model saved as {model_filename}')
    return epoch_loss, epoch_acc

def validate(epoch):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * inputs.size(0)
        writer.add_scalar('validation loss', loss.item(), epoch)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(val_set)
    epoch_acc = running_corrects.double() / len(val_set)
    writer.add_scalar('val loss', epoch_loss, epoch)
    writer.add_scalar('val acc', epoch_acc, epoch)
    print(f'val loss: {epoch_loss:.4f}, acc {epoch_acc:.4f}')
    return epoch_loss, epoch_acc

# Train and validate
epochs = 30
for epoch in range(epochs):
    train_loss, train_acc = train(epoch)
    val_loss, val_acc = validate(epoch)
    print('-' * 20)
writer.flush()
writer.close()


training loss: 0.5674, acc 0.7368
Model saved as model_epoch_0.pt
val loss: 0.8387, acc 0.5000
--------------------
training loss: 0.3170, acc 0.8037
Model saved as model_epoch_1.pt
val loss: 0.4698, acc 0.6875
--------------------
training loss: 0.1899, acc 0.9394
Model saved as model_epoch_2.pt
val loss: 0.8454, acc 0.6250
--------------------
training loss: 0.1453, acc 0.9486
Model saved as model_epoch_3.pt
val loss: 0.4468, acc 0.7500
--------------------
training loss: 0.1242, acc 0.9574
Model saved as model_epoch_4.pt
val loss: 0.7371, acc 0.6875
--------------------
training loss: 0.1291, acc 0.9565
Model saved as model_epoch_5.pt
val loss: 0.5150, acc 0.6875
--------------------
training loss: 0.1075, acc 0.9613
Model saved as model_epoch_6.pt
val loss: 0.6101, acc 0.7500
--------------------
training loss: 0.0973, acc 0.9670
Model saved as model_epoch_7.pt
val loss: 0.5851, acc 0.7500
--------------------
training loss: 0.0944, acc 0.9664
Model saved as model_epoch_8.pt
val lo

In [ ]:
# Define Optimizer and Loss Function
writer = SummaryWriter()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=model.to(device)

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters())
# Train the model
epochs = 10
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in train_loader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        writer.add_scalar('training loss',
                        running_loss / print_every,
                        steps)
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            writer.add_scalar('validation loss',
                            test_loss / len(val_loader),
                            steps)
            writer.add_scalar('accuracy',
                            accuracy / len(val_loader),
                            steps)
            train_losses.append(running_loss / len(train_loader))
            test_losses.append(test_loss / len(val_loader))
            print(f"Epoch {epoch+1}/{epochs}.. "
                f"Train loss: {running_loss / print_every:.3f}.. "
                f"Test loss: {test_loss / len(val_loader):.3f}.. "
                f"Test accuracy: {accuracy / len(val_loader):.3f}")
            running_loss = 0
            model.train()